In [28]:
import cv2
import os
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
#from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import Image

In [33]:
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.preprocessing import image

# Define the base directory
base_dir = "C:/Users/saitu/Downloads/AIOralCancerProject/AIOralCancerProject"

# Create an empty list to store the images and labels
X = []
y = []

# Get the class labels from the directory names
class_labels = os.listdir(base_dir)

# Iterate through each directory and store the images and labels
for i, label in enumerate(class_labels):
    img_dir = os.path.join(base_dir, label)
    for file in os.listdir(img_dir):
        # Check if file is an image
        if file.endswith(".jpg") or file.endswith(".png"):
            # Load the image
            
            img = Image.open(os.path.join(img_dir, file))
            img = img.resize((300, 300))
            img = np.array(img)
            #img = img / 255.0

            # Append the image to the list of images
            X.append(img)
            # Append the corresponding label to the list of labels
            y.append(i)

In [34]:
# Shuffle the data
#X, y = shuffle(X, y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [35]:
X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = X_train.reshape(-1,300,300,3)
X_test = X_test.reshape(-1,300,300,3)

In [37]:
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load the EfficientNetB3 model
base_model = EfficientNetB3(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

# Freeze the base model layers
base_model.trainable = False

# Create a new model on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
#x = Dropout(0.5)(x)
predictions = Dense(16, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train_encoded, batch_size=64, epochs=30, validation_data=(X_test, y_test_encoded))

Epoch 1/30
35/35 [==============================] - 284s 8s/step - loss: 2.6860 - accuracy: 0.1308 - val_loss: 2.6750 - val_accuracy: 0.1259
Epoch 2/30
35/35 [==============================] - 287s 8s/step - loss: 2.5345 - accuracy: 0.1614 - val_loss: 2.6617 - val_accuracy: 0.1439
Epoch 3/30
35/35 [==============================] - 397s 11s/step - loss: 2.4716 - accuracy: 0.1785 - val_loss: 2.6537 - val_accuracy: 0.1439
Epoch 4/30
35/35 [==============================] - 271s 8s/step - loss: 2.4131 - accuracy: 0.1983 - val_loss: 2.6767 - val_accuracy: 0.1259
Epoch 5/30
35/35 [==============================] - 265s 8s/step - loss: 2.3603 - accuracy: 0.2100 - val_loss: 2.6732 - val_accuracy: 0.1367
Epoch 6/30
35/35 [==============================] - 1972s 58s/step - loss: 2.3259 - accuracy: 0.2262 - val_loss: 2.6813 - val_accuracy: 0.1385
Epoch 7/30
35/35 [==============================] - 179s 5s/step - loss: 2.2908 - accuracy: 0.2293 - val_loss: 2.7006 - val_accuracy: 0.1331
Epoch 8/30

In [38]:
import tensorflow as tf
tf.saved_model.save(model, 'C:/Users/saitu/Downloads/AIOralCancerProject')

INFO:tensorflow:Assets written to: C:/Users/saitu/Downloads/AIOralCancerProject\assets


INFO:tensorflow:Assets written to: C:/Users/saitu/Downloads/AIOralCancerProject\assets


In [39]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test_encoded)
print('Test accuracy:', test_acc)

18/18 [==============================] - 34s 2s/step - loss: 2.8408 - accuracy: 0.1133
Test accuracy: 0.1133093535900116
